# Module 05: Interactive Plotly Visualizations

**Estimated Time**: 90 minutes  
**Difficulty**: Intermediate

## Learning Objectives

By the end of this module, you will:
- Understand Plotly Express vs Graph Objects
- Create interactive line, scatter, and bar charts
- Implement hover tooltips and custom interactions
- Build animations with sliders and play buttons
- Create 3D visualizations
- Design interactive dashboards
- Export visualizations to HTML

---

In [ ]:
# Import required libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set random seed
np.random.seed(42)

print("Libraries imported successfully!")
print(f"Plotly version: {px.__version__ if hasattr(px, '__version__') else 'N/A'}")
print("\nPlotly creates interactive visualizations that work in:")
print("  - Jupyter notebooks")
print("  - Web browsers (standalone HTML)")
print("  - Dashboards (Dash framework)")

## Part 1: Plotly Express vs Graph Objects

Plotly has two main interfaces:

### Plotly Express (px) - High-level, Easy
- **Pros**: Simple syntax, quick plots, sensible defaults
- **Cons**: Less customization control
- **Use for**: Exploratory analysis, standard plots, quick visualizations

### Graph Objects (go) - Low-level, Powerful
- **Pros**: Complete control, advanced features, custom layouts
- **Cons**: More verbose, steeper learning curve
- **Use for**: Complex visualizations, custom interactivity, dashboards

### Rule of Thumb
Start with Express, switch to Graph Objects when you need more control.

In [ ]:
# Create sample data
df = pd.DataFrame(
    {
        "x": np.arange(50),
        "y": np.random.randn(50).cumsum(),
        "category": np.random.choice(["A", "B", "C"], 50),
    }
)

print("Sample data:")
print(df.head())

In [ ]:
# Example 1: Same plot with Express vs Graph Objects

# Plotly Express (3 lines)
fig_express = px.line(df, x="x", y="y", title="Plotly Express - Simple and Fast")
fig_express.show()

print("\nPlotly Express: 3 lines of code, automatic styling!")

In [ ]:
# Graph Objects (more lines, more control)
fig_go = go.Figure()

fig_go.add_trace(
    go.Scatter(x=df["x"], y=df["y"], mode="lines", name="Data", line=dict(color="blue", width=2))
)

fig_go.update_layout(
    title="Graph Objects - More Control",
    xaxis_title="X values",
    yaxis_title="Y values",
    hovermode="x unified",
)

fig_go.show()

print("\nGraph Objects: More code, but precise control over every element!")

## Part 2: Interactive Line and Scatter Plots

The power of Plotly is interactivity: hover, zoom, pan, select, and more.

In [ ]:
# Create time series data
dates = pd.date_range(start="2024-01-01", periods=365, freq="D")
temperature = 15 * np.sin(2 * np.pi * np.arange(365) / 365) + 20 + np.random.normal(0, 2, 365)
humidity = np.random.uniform(40, 80, 365)

weather_df = pd.DataFrame(
    {"date": dates, "temperature": temperature, "humidity": humidity, "month": dates.month_name()}
)

print("Weather data created")
print(weather_df.head())

In [ ]:
# Example 1: Interactive line plot with hover data
fig = px.line(
    weather_df,
    x="date",
    y="temperature",
    title="Interactive Temperature Plot",
    labels={"temperature": "Temperature (°C)", "date": "Date"},
)

# Customize hover template
fig.update_traces(
    hovertemplate="<b>Date:</b> %{x}<br>" + "<b>Temperature:</b> %{y:.1f}°C<br>" + "<extra></extra>"
)

# Update layout
fig.update_layout(hovermode="x unified", height=500)

fig.show()

print("\nInteractive features to try:")
print("  - Hover over points to see details")
print("  - Click and drag to zoom")
print("  - Double-click to reset")
print("  - Click legend items to show/hide")
print("  - Use toolbar icons for pan, zoom, etc.")

In [ ]:
# Example 2: Multi-line interactive plot
fig = go.Figure()

# Add temperature line
fig.add_trace(
    go.Scatter(
        x=weather_df["date"],
        y=weather_df["temperature"],
        mode="lines",
        name="Temperature",
        line=dict(color="orangered", width=2),
        hovertemplate="Temp: %{y:.1f}°C<extra></extra>",
    )
)

# Add 7-day moving average
ma_7 = weather_df["temperature"].rolling(window=7).mean()
fig.add_trace(
    go.Scatter(
        x=weather_df["date"],
        y=ma_7,
        mode="lines",
        name="7-day MA",
        line=dict(color="blue", width=2, dash="dash"),
        hovertemplate="7-day MA: %{y:.1f}°C<extra></extra>",
    )
)

# Update layout
fig.update_layout(
    title="Temperature with Moving Average",
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    hovermode="x unified",
    height=500,
    showlegend=True,
)

fig.show()

In [ ]:
# Example 3: Interactive scatter plot with custom hover
# Create scatter data
scatter_df = pd.DataFrame(
    {
        "temperature": weather_df["temperature"],
        "humidity": weather_df["humidity"],
        "month": weather_df["month"],
        "season": pd.cut(
            weather_df["date"].dt.month,
            bins=[0, 3, 6, 9, 12],
            labels=["Winter", "Spring", "Summer", "Fall"],
        ),
    }
)

fig = px.scatter(
    scatter_df,
    x="temperature",
    y="humidity",
    color="season",
    title="Temperature vs Humidity by Season",
    labels={"temperature": "Temperature (°C)", "humidity": "Humidity (%)"},
    opacity=0.6,
    size_max=10,
)

fig.update_traces(marker=dict(size=8))

fig.update_layout(height=600, hovermode="closest")

fig.show()

print("\nTry clicking legend items to isolate seasons!")

## Part 3: Interactive Bar Charts and Histograms

In [ ]:
# Create sales data
sales_df = pd.DataFrame(
    {
        "month": [
            "Jan",
            "Feb",
            "Mar",
            "Apr",
            "May",
            "Jun",
            "Jul",
            "Aug",
            "Sep",
            "Oct",
            "Nov",
            "Dec",
        ],
        "product_a": np.random.randint(50, 100, 12),
        "product_b": np.random.randint(40, 90, 12),
        "product_c": np.random.randint(30, 80, 12),
    }
)

print("Sales data:")
print(sales_df)

In [ ]:
# Example 1: Interactive grouped bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        name="Product A",
        x=sales_df["month"],
        y=sales_df["product_a"],
        marker_color="#0173B2",
        hovertemplate="Product A<br>%{x}: %{y}K<extra></extra>",
    )
)

fig.add_trace(
    go.Bar(
        name="Product B",
        x=sales_df["month"],
        y=sales_df["product_b"],
        marker_color="#DE8F05",
        hovertemplate="Product B<br>%{x}: %{y}K<extra></extra>",
    )
)

fig.add_trace(
    go.Bar(
        name="Product C",
        x=sales_df["month"],
        y=sales_df["product_c"],
        marker_color="#029E73",
        hovertemplate="Product C<br>%{x}: %{y}K<extra></extra>",
    )
)

fig.update_layout(
    title="Monthly Sales by Product",
    xaxis_title="Month",
    yaxis_title="Sales (Thousands)",
    barmode="group",
    height=500,
    hovermode="x unified",
)

fig.show()

In [ ]:
# Example 2: Stacked bar chart with percentage
fig = go.Figure()

fig.add_trace(
    go.Bar(name="Product A", x=sales_df["month"], y=sales_df["product_a"], marker_color="lightblue")
)

fig.add_trace(
    go.Bar(name="Product B", x=sales_df["month"], y=sales_df["product_b"], marker_color="coral")
)

fig.add_trace(
    go.Bar(
        name="Product C", x=sales_df["month"], y=sales_df["product_c"], marker_color="lightgreen"
    )
)

fig.update_layout(
    title="Stacked Sales by Product",
    xaxis_title="Month",
    yaxis_title="Total Sales (Thousands)",
    barmode="stack",
    height=500,
)

fig.show()

print("\nClick legend items to show/hide products!")

In [ ]:
# Example 3: Interactive histogram
# Generate random data
data1 = np.random.normal(100, 15, 1000)
data2 = np.random.normal(110, 12, 1000)

fig = go.Figure()

fig.add_trace(go.Histogram(x=data1, name="Group A", opacity=0.7, marker_color="blue", nbinsx=30))

fig.add_trace(go.Histogram(x=data2, name="Group B", opacity=0.7, marker_color="red", nbinsx=30))

fig.update_layout(
    title="Interactive Histogram Comparison",
    xaxis_title="Value",
    yaxis_title="Frequency",
    barmode="overlay",
    height=500,
)

fig.show()

## Part 4: Animations and Sliders

Animations add a time dimension to your visualizations.

In [ ]:
# Create animated data
years = list(range(2020, 2025))
countries = ["USA", "China", "India", "Japan", "Germany"]

data_list = []
for year in years:
    for country in countries:
        data_list.append(
            {
                "year": year,
                "country": country,
                "gdp": np.random.uniform(1, 20) * (1 + (year - 2020) * 0.05),
                "population": np.random.uniform(50, 1400),
            }
        )

animated_df = pd.DataFrame(data_list)

print("Animated data created:")
print(animated_df.head())

In [ ]:
# Example 1: Animated scatter plot
fig = px.scatter(
    animated_df,
    x="gdp",
    y="population",
    animation_frame="year",
    animation_group="country",
    size="gdp",
    color="country",
    hover_name="country",
    size_max=60,
    range_x=[0, 25],
    range_y=[0, 1500],
    title="GDP vs Population Over Time (Animated)",
)

fig.update_layout(xaxis_title="GDP (Trillion USD)", yaxis_title="Population (Millions)", height=600)

fig.show()

print("\nClick the PLAY button to see animation!")
print("Use the slider to navigate through years.")

In [ ]:
# Example 2: Animated bar chart race
fig = px.bar(
    animated_df,
    x="gdp",
    y="country",
    animation_frame="year",
    orientation="h",
    color="country",
    range_x=[0, 25],
    title="GDP Race by Country",
)

fig.update_layout(
    xaxis_title="GDP (Trillion USD)", yaxis_title="Country", height=500, showlegend=False
)

fig.show()

print("\nBar chart race - watch countries compete over time!")

In [ ]:
# Example 3: Custom slider for parameter exploration
x = np.linspace(0, 10, 100)

# Create figure
fig = go.Figure()

# Add traces for different frequencies
for freq in [1, 2, 3, 4, 5]:
    fig.add_trace(
        go.Scatter(
            visible=False, x=x, y=np.sin(freq * x), name=f"Frequency={freq}", line=dict(width=2)
        )
    )

# Make first trace visible
fig.data[0].visible = True

# Create slider steps
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}, {"title": f"Sine Wave (Frequency = {i+1})"}],
        label=str(i + 1),
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

# Create slider
sliders = [
    dict(
        active=0,
        yanchor="top",
        y=0.1,
        xanchor="left",
        currentvalue={"prefix": "Frequency: ", "visible": True, "xanchor": "right"},
        pad={"b": 10, "t": 50},
        len=0.9,
        x=0.1,
        steps=steps,
    )
]

fig.update_layout(
    sliders=sliders,
    title="Sine Wave (Frequency = 1)",
    xaxis_title="X",
    yaxis_title="sin(frequency × x)",
    height=500,
)

fig.show()

print("\nUse the slider to change the sine wave frequency!")

## Part 5: 3D Visualizations

Plotly excels at interactive 3D plots.

In [ ]:
# Example 1: 3D Scatter plot
# Generate 3D data
n_points = 500
df_3d = pd.DataFrame(
    {
        "x": np.random.randn(n_points),
        "y": np.random.randn(n_points),
        "z": np.random.randn(n_points),
        "category": np.random.choice(["A", "B", "C"], n_points),
    }
)

fig = px.scatter_3d(
    df_3d,
    x="x",
    y="y",
    z="z",
    color="category",
    title="Interactive 3D Scatter Plot",
    opacity=0.7,
    size_max=10,
)

fig.update_traces(marker=dict(size=4))

fig.update_layout(
    scene=dict(xaxis_title="X Axis", yaxis_title="Y Axis", zaxis_title="Z Axis"), height=700
)

fig.show()

print("\n3D Interaction tips:")
print("  - Click and drag to rotate")
print("  - Scroll to zoom")
print("  - Shift+drag to pan")

In [ ]:
# Example 2: 3D Surface plot
# Create surface data
x = np.linspace(-5, 5, 50)
y = np.linspace(-5, 5, 50)
X, Y = np.meshgrid(x, y)
Z = np.sin(np.sqrt(X**2 + Y**2))

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale="Viridis")])

fig.update_layout(
    title="3D Surface Plot: sin(√(x² + y²))",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.3)),
    ),
    height=700,
)

fig.show()

print("\nRotate the surface to see the 3D structure!")

In [ ]:
# Example 3: 3D Line plot (trajectory)
t = np.linspace(0, 20, 1000)
x = np.sin(t)
y = np.cos(t)
z = t

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode="lines",
            line=dict(color=z, colorscale="Plasma", width=5),
            marker=dict(size=2),
        )
    ]
)

fig.update_layout(
    title="3D Helix Trajectory",
    scene=dict(xaxis_title="X (sin(t))", yaxis_title="Y (cos(t))", zaxis_title="Z (t)"),
    height=700,
)

fig.show()

## Part 6: Subplots and Dashboards

In [ ]:
# Example 1: Multiple subplots
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("Line Plot", "Scatter Plot", "Bar Chart", "Histogram"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}], [{"type": "bar"}, {"type": "histogram"}]],
)

# Line plot
x = np.linspace(0, 10, 50)
fig.add_trace(go.Scatter(x=x, y=np.sin(x), name="sin(x)", line=dict(color="blue")), row=1, col=1)

# Scatter plot
fig.add_trace(
    go.Scatter(
        x=np.random.randn(50),
        y=np.random.randn(50),
        mode="markers",
        name="Random",
        marker=dict(color="red"),
    ),
    row=1,
    col=2,
)

# Bar chart
fig.add_trace(
    go.Bar(
        x=["A", "B", "C", "D"], y=[20, 14, 23, 18], name="Categories", marker=dict(color="green")
    ),
    row=2,
    col=1,
)

# Histogram
fig.add_trace(
    go.Histogram(x=np.random.normal(0, 1, 1000), name="Distribution", marker=dict(color="purple")),
    row=2,
    col=2,
)

fig.update_layout(
    title_text="Interactive Dashboard with Multiple Plot Types", showlegend=False, height=700
)

fig.show()

In [ ]:
# Example 2: Financial dashboard
# Create stock data
dates = pd.date_range(start="2024-01-01", periods=100, freq="D")
price = 100 + np.random.randn(100).cumsum()
volume = np.random.randint(1000000, 5000000, 100)

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.5, 0.25, 0.25],
    subplot_titles=("Price", "Volume", "Returns"),
)

# Price
fig.add_trace(
    go.Scatter(x=dates, y=price, name="Price", line=dict(color="navy", width=2)), row=1, col=1
)

# Moving average
ma = pd.Series(price).rolling(window=20).mean()
fig.add_trace(
    go.Scatter(x=dates, y=ma, name="20-day MA", line=dict(color="orange", width=2, dash="dash")),
    row=1,
    col=1,
)

# Volume
fig.add_trace(
    go.Bar(x=dates, y=volume, name="Volume", marker=dict(color="steelblue")), row=2, col=1
)

# Returns
returns = np.diff(price) / price[:-1] * 100
colors = ["green" if r > 0 else "red" for r in returns]
fig.add_trace(
    go.Bar(x=dates[1:], y=returns, name="Returns", marker=dict(color=colors)), row=3, col=1
)

fig.update_layout(
    title_text="Stock Analysis Dashboard", height=900, showlegend=True, hovermode="x unified"
)

fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="Returns (%)", row=3, col=1)

fig.show()

print("\nInteractive financial dashboard!")
print("Try zooming on one subplot - all subplots zoom together!")

## Part 7: Exporting Interactive Visualizations

In [ ]:
# Example: Export to HTML
# Create a sample interactive plot
fig = px.scatter(
    weather_df,
    x="temperature",
    y="humidity",
    color="month",
    size_max=10,
    title="Temperature vs Humidity (Interactive)",
    labels={"temperature": "Temperature (°C)", "humidity": "Humidity (%)"},
)

fig.update_traces(marker=dict(size=8))

# Save to HTML file
output_path = "../notebooks/outputs/interactive_plot.html"
fig.write_html(output_path)

fig.show()

print(f"\nInteractive plot saved to: {output_path}")
print("\nThis HTML file can be:")
print("  - Opened in any web browser")
print("  - Shared with others (single file, no dependencies)")
print("  - Embedded in websites")
print("  - Included in reports")
print("\nFull interactivity preserved!")

In [ ]:
# Additional export options
print("Plotly Export Formats:")
print("\n1. HTML (Interactive):")
print("   fig.write_html('plot.html')")
print("   - Full interactivity")
print("   - Self-contained")
print("   - Works offline")

print("\n2. Static Images (requires kaleido):")
print("   fig.write_image('plot.png')")
print("   fig.write_image('plot.pdf')")
print("   fig.write_image('plot.svg')")
print("   - High resolution")
print("   - For publications")

print("\n3. JSON (for programmatic use):")
print("   fig.write_json('plot.json')")
print("   - Full plot configuration")
print("   - Can be loaded later")

print("\n4. Online (Plotly Chart Studio):")
print("   import plotly.io as pio")
print("   pio.write_html(fig, 'plot.html')")
print("   - Share via URL")
print("   - Collaborative editing")

## Part 8: Key Takeaways

### What You've Learned
✓ **Plotly Express**: Quick, beautiful interactive plots  
✓ **Graph Objects**: Full control over every element  
✓ **Interactivity**: Hover, zoom, pan, select, click  
✓ **Line & Scatter**: Interactive temporal and relationship plots  
✓ **Bar & Histogram**: Categorical and distribution visualizations  
✓ **Animations**: Time-based animations with sliders  
✓ **3D Plots**: Interactive 3D scatter, surface, and line plots  
✓ **Dashboards**: Multi-panel interactive dashboards  
✓ **Export**: HTML files with full interactivity  

### When to Use Plotly

**Perfect for:**
- Exploratory data analysis (zoom and pan)
- Presentations (interactive demos)
- Web dashboards
- Large datasets (better than static plots)
- 3D visualizations
- Sharing with non-technical audiences

**Not ideal for:**
- Printed publications (use Matplotlib)
- When file size matters (HTML can be large)
- Simple static plots (overkill)

### Plotly vs Matplotlib

| Feature | Plotly | Matplotlib |
|---------|--------|------------|
| Interactivity | Excellent | Limited |
| Ease of use | Easy (Express) | Moderate |
| Customization | Good | Excellent |
| 3D plots | Excellent | Good |
| File output | HTML, PNG | PNG, PDF, SVG |
| Learning curve | Gentle | Steeper |
| Best for | Web, dashboards | Publications |

### What's Next
In **Module 06**, you'll learn scientific best practices:
- Publication-quality figures
- DPI and sizing for different media
- Color theory and accessibility
- Common visualization mistakes
- Choosing the right chart type
- Data-ink ratio principles

---

## Exercises

Practice your Plotly skills!

### Exercise 1: Interactive Time Series
1. Create daily stock price data for 1 year
2. Build an interactive plot with:
   - Price line
   - 20-day and 50-day moving averages
   - Custom hover showing date, price, and percentage change
   - Range selector buttons (1M, 3M, 6M, 1Y, All)
3. Export to HTML

In [ ]:
# Your code here

### Exercise 2: Animated Data Exploration
1. Create a dataset with multiple countries and years
2. Include variables like GDP, population, life expectancy
3. Create an animated bubble chart showing:
   - GDP vs life expectancy
   - Bubble size = population
   - Color = continent/region
   - Animation = year
4. Add play/pause controls

In [ ]:
# Your code here

### Exercise 3: 3D Data Visualization
1. Create 3D data representing a function: z = f(x, y)
2. Create two visualizations:
   - 3D surface plot with custom colorscale
   - 3D scatter plot of sample points
3. Add appropriate titles and axis labels
4. Customize the camera angle for best view

In [ ]:
# Your code here

### Exercise 4: Interactive Dashboard
Create a sales analytics dashboard with 4 subplots:
1. Monthly sales trend (line)
2. Sales by product category (bar)
3. Sales distribution (histogram)
4. Geographic sales (if you want, create a simple scatter representing regions)

Requirements:
- Share x-axis where appropriate
- Unified hover mode
- Consistent color scheme
- Professional titles and labels

In [ ]:
# Your code here

### Challenge: Build a Complete Interactive Report
Create a comprehensive interactive report analyzing weather data:

1. **Overview Section**:
   - Summary statistics in text
   - Key findings

2. **Interactive Visualizations**:
   - Temperature over time with moving averages
   - Temperature vs humidity scatter (colored by season)
   - Monthly temperature distribution (box plots)
   - Animated yearly comparison (if multi-year data)

3. **Export**:
   - Save as standalone HTML report
   - Include all interactivity
   - Professional styling

4. **Bonus**:
   - Add dropdown to select different metrics
   - Include annotations for extreme events
   - Custom color scheme matching brand colors

In [ ]:
# Your code here

---

**Congratulations!** You've mastered interactive visualizations with Plotly. You can now create engaging, interactive plots that allow users to explore data dynamically.

**Next**: Module 06 - Scientific Best Practices for Publication-Quality Visualizations